In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, sys
PATH = os.path.abspath("")[:-3]

In [3]:

# SCRAPPEAR UN USUARIO
from test_user import get_user_data

USER = "ComicGirlAshley"
user_data = get_user_data(USER)
print(user_data)

[245, False, 26735, 99, 20170, 134, 6313, 5.049019607843137, 16.11764705882353, 353.62745098039215, 0.8627450980392157, 0.1568627450980392, 26636]


c:\Users\maria\Documents\Uni\ia orgs\bot_detector\.venv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
# Se leen los datos generados con model.py\n",
df_data = pd.read_csv(PATH + "data/training_new.csv").sample(frac=1).reset_index(drop=True)
data = df_data.iloc[:, 1:]
print(data.head())

   factor_rep  verified  followersCount  friendsCount  tweetsCount  \
0         298         0            1637          2062          943   
1         222         1          455930             0           85   
2         279         0             175           608         1986   
3         193         0             660          3301          504   
4         775         0            6923          6107         1236   

   listedCount  mediaCount  avg_reply  avg_retweet  like_reply  avg_quote  \
0            5         115   0.520833     1.625000   22.604167   0.145833   
1         7649          83   0.000000     2.560000   19.560000   0.280000   
2            0          57   0.340000     0.100000    1.000000   0.020000   
3            0          36   1.921053     0.105263    4.105263   0.026316   
4           35        1012   0.096154     0.538462    0.711538   0.038462   

   hora_rep  followers_diff  clase  
0  0.187500           -1637    bot  
1  0.200000         -455929  human  
2  0.

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
header_names = list(data.columns.values)
for col in data.iloc[:,:-1]:
    data[col] = preprocessing.MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))
x_data = data.iloc[:,:-1].to_numpy()
y_data = data.iloc[:,-1].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, stratify=y_data, test_size=0.33)

In [6]:
#CONVERTIRLA SALIDA A UN VECTOR DE 0 Y 1
from sklearn.preprocessing import LabelBinarizer

encoder=LabelBinarizer()
label = encoder.fit_transform(y_train)
y_train_transformed = np.hstack((label, 1 - label))
label = encoder.fit_transform(y_test)
y_test_transformed = np.hstack((label, 1 - label))
print(y_train[:5])
print(y_train_transformed[:5])

['bot' 'bot' 'bot' 'human' 'human']
[[0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]]


In [7]:
# COMPROBAR TIPO DE LOS DATOS
print(type(x_train))
print(type(y_train_transformed))
# COMPROBAR DIMENSIONES DE LOS DATOS
print(x_train.shape)
print(y_train.shape)
print(y_train_transformed.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test_transformed.shape) 
# OBTENER DIMENSION DE LA ENTRADA Y NÚMERO DE SALIDAS
input_shape = (x_train.shape[1] ,)
num_clases = y_test_transformed.shape[1]

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1616, 13)
(1616,)
(1616, 2)
(796, 13)
(796,)
(796, 2)


# Buscar modelo

In [8]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, 
    hidden_layer_sizes=(20, 20), random_state=1)

clf.fit(x_train, y_train)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 20), random_state=1,
              solver='lbfgs')
clf.predict([[2., 2.], [-1., -2.]])

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20, 20), random_state=1,
              solver='lbfgs')

# Test

In [82]:
from keras.models import Sequential
from keras.layers import Dense
input_shape= (x_train.shape[1] ,)
final_model = Sequential()
arch = [20, 20, 0, "relu", 0.2]
final_model.add(Dense(arch[0], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(arch[1], input_shape=input_shape, activation=arch[3]))
final_model.add(Dense(2, activation='softmax'))
final_model.summary()

# CONFIGURAR MODELO Y ENTRENAMIENTO
from keras.callbacks import EarlyStopping

epochs = 20
final_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=arch[4], momentum=0), metrics=['accuracy','mse'], loss='mean_squared_error')
historico = final_model.fit(x_train, y_train_transformed, epochs=epochs, verbose=1, shuffle=False, batch_size=16)

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_84 (Dense)            (None, 20)                280       
                                                                 
 dense_85 (Dense)            (None, 20)                420       
                                                                 
 dense_86 (Dense)            (None, 2)                 42        
                                                                 
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
101/101 [==============================] - 0s 1ms/step - loss: 0.2122 - accuracy: 0.6906 - mse: 0.2122
Epoch 2/20
101/101 [==============================] - 0s 1ms/step - loss: 0.1946 - accuracy: 0.7073 - mse: 0.1946
Epoch 3/20
101/101 [==============================] - 0s 1ms/step - loss: 0.1931 - accuracy

In [83]:
# EVALUAR MODELO DEFINITIVO
train_results = final_model.evaluate(x_train, y_train_transformed, verbose=1)
test_results= final_model.evaluate(x_test, y_test_transformed, verbose=1)
#EL INDICE 0 ES EL LOSS. EL RESTO, LAS MÉTRICAS QUE SE HAN ESPECIFICADO AL COMPILAR EL MODELO.
# #EN ESTE CASO 'accuracy':1,'mse':2
print(f'Train results-Loss: {train_results[0]} -Accuracy: {train_results[1]} -MSE: {train_results[2]}')
print(f'Test results-Loss: {test_results[0]} -Accuracy: {test_results[1]} -MSE: {test_results[2]}')

25/25 [==============================] - 0s 1ms/step - loss: 0.1885 - accuracy: 0.6960 - mse: 0.1885
Train results-Loss: 0.18516811728477478 -Accuracy: 0.7035890817642212 -MSE: 0.18516811728477478
Test results-Loss: 0.1885344386100769 -Accuracy: 0.69597989320755 -MSE: 0.1885344386100769


In [84]:
# PREDICCIONES DE LAS CLASES
# PREDICCIONES EN BRUTO
raw_testPred= final_model.predict(x_test)
print(raw_testPred[:5])

# PREDICCIONES DE LA CLASE
testPred= np.argmax(raw_testPred, axis=1)
#TRANSFORMAR EL NÚMERO DE COLUMNA EN LA ETIQUETA
class_testPred= encoder.classes_[testPred]
print(class_testPred[:5])

25/25 [==============================] - 0s 1ms/step
[[0.36264595 0.6373541 ]
 [0.91705054 0.08294941]
 [0.38812006 0.61187994]
 [0.42710605 0.572894  ]
 [0.36394072 0.6360593 ]]
['human' 'bot' 'human' 'human' 'human']


In [87]:
test_targets = y_test
print(class_testPred[:6])
print(test_targets[:6])
comp_df = pd.DataFrame({"predicción": class_testPred, 'valor real': test_targets}, columns=['predicción','valor real'])
#guardar en fichero las salidas de test y target
comp_df.to_csv(PATH + 'data/output/salidas_test.csv', index=False)
#mostrar 10 primeras filas
print(comp_df.iloc[:10,:])

['human' 'bot' 'human' 'human' 'human' 'human']
['bot' 'human' 'human' 'human' 'human' 'bot']
  predicción valor real
0      human        bot
1        bot      human
2      human      human
3      human      human
4      human      human
5      human        bot
6      human      human
7      human        bot
8        bot      human
9      human        bot


In [88]:
# MATRIZ DE CONFUSIÓN
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,class_testPred)
print('Matriz de confusión')
print(cm)
print('Classification Report')
print(classification_report(y_test,class_testPred))

Matriz de confusión
[[ 20 444]
 [110 222]]
Classification Report
              precision    recall  f1-score   support

         bot       0.15      0.04      0.07       464
       human       0.33      0.67      0.44       332

    accuracy                           0.30       796
   macro avg       0.24      0.36      0.26       796
weighted avg       0.23      0.30      0.22       796

